In [5]:
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2

In [6]:
#data train network
net = cv2.dnn.readNetFromCaffe('deploy.prototxt.txt', 'res10_300x300_ssd_iter_140000.caffemodel')

In [7]:
#stream video
stream = VideoStream(src=0).start()
time.sleep(2.0)

#frame
while True:
    frame = stream.read()
    frame = imutils.resize(frame, width=400)
    
    #convert to blob
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,(300, 300), (104.0, 177.0, 123.0))
    
    #prediksi
    net.setInput(blob)
    detection = net.forward()
    
    #deteksi 
    for i in range(0, detection.shape[2]):
        confidence = detection[0, 0, i, 2]
        
        if confidence < 0.5:
            continue
            
        #posisi box
        box = detection[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        
        #bounding objek
        text = "{:.2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        
        cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 0, 255), 2)
        cv2.putText(frame, text, (startX, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
        
    #output
    cv2.imshow("Frame", frame)
    
    #shoot
    if cv2.waitKey(1) & 0xFF == ord('s'):
        cv2.imwrite("frames.jpg", frame)
    
    # q for quit
    elif cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    
            
cv2.destroyAllWindows()
stream.stop()